In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy

In [ ]:
import os,cv2
directory='../input/ckdataset/CK+_Complete'
dir_list=os.listdir(directory)
#print(dir_list)
image_data_list=[]
labels=[]
emotions=[]
l=0
for data_dir in dir_list:
    data_list=os.listdir(directory+'/'+data_dir)
    print(data_dir)
    for img in data_list:
        input_img=cv2.imread(directory+'/'+data_dir+'/'+img)
        input_img=cv2.resize(input_img,(224, 224))
        image_data_list.append(input_img)
        labels.append(l)
    emotions.append(data_dir)
    l=l+1
img_data=np.array(image_data_list)
img_data=img_data/255
img_data.shape

for i in range(7):
    print('No of instances with class value '+ str(i) + ' : ' + str(labels.count(i)))

In [ ]:
num_classes=7
labels_array = np.asarray(labels)
# Y = keras.utils.to_categorical(labels, num_classes)
X_train, X_t, y_train, y_t = train_test_split(img_data, labels_array, test_size=0.2,shuffle= True, random_state=20) 
X_val,X_test,y_val,y_test = train_test_split(X_t,y_t,test_size=0.9,shuffle= False)

print(y_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
from keras.applications.vgg16 import VGG16

image_model = VGG16(weights='../input/placesvgg16/vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224, 224, 3))
for layer in image_model.layers[:15]:
    layer.trainable = False

In [ ]:
print(np.expand_dims(X_train[0], 0).shape)
image_model.predict(np.expand_dims(X_train[1243], 0))


In [ ]:
import tensorflow as tf
x = image_model.output
print(x.shape)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dense(num_classes, activation='softmax')(x)
transfer_model = keras.Model(inputs=image_model.input, outputs=x)
transfer_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

In [ ]:
history=transfer_model.fit(X_train,y_train,epochs=128,validation_data=(X_val,y_val))

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (15,7)
plt.rcParams['font.size']=15
print(history.history.keys())
#  "Accuracy"
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.savefig("accuracy.jpg")

plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.savefig("loss.jpg")

plt.show()

In [ ]:
transfer_model.evaluate(X_test, y_test)